In [7]:
import pandas as pd
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, value
import csv

In [3]:
K = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M"]
I = ["Aircraft", "Fuel", "Employee"]
J = ["Passenger", "Freight"]

# Parameters building
X = { 
    i: {
        k: 0 for k in K
    } for i in I
}
Y = { 
    j: {
        k: 0 for k in K
    } for j in J
}

In [4]:
X

{'Aircraft': {'A': 0,
  'B': 0,
  'C': 0,
  'D': 0,
  'E': 0,
  'F': 0,
  'G': 0,
  'H': 0,
  'I': 0,
  'J': 0,
  'K': 0,
  'L': 0,
  'M': 0},
 'Fuel': {'A': 0,
  'B': 0,
  'C': 0,
  'D': 0,
  'E': 0,
  'F': 0,
  'G': 0,
  'H': 0,
  'I': 0,
  'J': 0,
  'K': 0,
  'L': 0,
  'M': 0},
 'Employee': {'A': 0,
  'B': 0,
  'C': 0,
  'D': 0,
  'E': 0,
  'F': 0,
  'G': 0,
  'H': 0,
  'I': 0,
  'J': 0,
  'K': 0,
  'L': 0,
  'M': 0}}

In [8]:
with open('../data/airlines_data.csv', newline='') as csvfile:
    rows = csv.DictReader(csvfile)
    k = 0
    for row in rows:
        for i in I:
            X[i][K[k]] = float(row[i]) 
        for j in J:
            Y[j][K[k]] = float(row[j])
        k += 1

In [9]:
df = pd.read_csv("../data/airlines_data.csv")

In [10]:
df

,Aircraft,Fuel,Employee,Passenger,Freight
0,109,392,8259,23756,870
1,115,381,9628,24183,1359
2,767,2673,70923,163483,12449
3,90,282,9683,10370,509
4,461,1608,40630,99047,3726
5,628,2074,47420,128635,9214
6,81,75,7115,11962,536
7,153,458,10177,32436,1462
8,455,1722,29124,83862,6337
9,103,400,8987,14618,785


In [11]:
model = LpProblem('CRS_model', LpMinimize)

In [12]:
theta_r = LpVariable(f'theta_r')
lambda_k = LpVariable.dicts(f'lambda_k', lowBound=0, indexs=K)

C:\Users\duclh3\AppData\Local\Temp\ipykernel_19348\3699632885.py:2: DeprecationWarning: 'indexs' is deprecated; use 'indices'.
  lambda_k = LpVariable.dicts(f'lambda_k', lowBound=0, indexs=K)


In [14]:
model += theta_r

In [15]:
lambda_k

{'A': lambda_k_A,
 'B': lambda_k_B,
 'C': lambda_k_C,
 'D': lambda_k_D,
 'E': lambda_k_E,
 'F': lambda_k_F,
 'G': lambda_k_G,
 'H': lambda_k_H,
 'I': lambda_k_I,
 'J': lambda_k_J,
 'K': lambda_k_K,
 'L': lambda_k_L,
 'M': lambda_k_M}

In [16]:
df

,Aircraft,Fuel,Employee,Passenger,Freight
0,109,392,8259,23756,870
1,115,381,9628,24183,1359
2,767,2673,70923,163483,12449
3,90,282,9683,10370,509
4,461,1608,40630,99047,3726
5,628,2074,47420,128635,9214
6,81,75,7115,11962,536
7,153,458,10177,32436,1462
8,455,1722,29124,83862,6337
9,103,400,8987,14618,785


In [38]:
# CRS_DEA_Model        
def getOverallEfficiency(r):

    # Model Building
    model = LpProblem('CRS_model', LpMinimize) # 建立一個新的model，命名為model
    
    # Decision variables Building
    theta_r = LpVariable(f'theta_r')
    lambda_k = LpVariable.dicts(f'lambda_k', lowBound=0, indexs=K)
    
    # Objective Function setting
    model += theta_r
    
    # Constraints setting
    for i in I:
        model += lpSum([
                lambda_k[k] * X[i][k]
            for k in K]) <= theta_r * float(X[i][K[r]])
    for j in J:
        model += lpSum([
                lambda_k[k] * Y[j][k]
            for k in K]) >= float(Y[j][K[r]])
    print(model)
    # Model solving
    model.solve()
    
    return f'{K[r]}: {round(value(model.objective), 3)}\n', value(model.objective)


In [40]:
getOverallEfficiency(1)

CRS_model:
MINIMIZE
1*theta_r + 0
SUBJECT TO
_C1: 109 lambda_k_A + 115 lambda_k_B + 767 lambda_k_C + 90 lambda_k_D
 + 461 lambda_k_E + 628 lambda_k_F + 81 lambda_k_G + 153 lambda_k_H
 + 455 lambda_k_I + 103 lambda_k_J + 547 lambda_k_K + 560 lambda_k_L
 + 423 lambda_k_M - 115 theta_r <= 0

_C2: 392 lambda_k_A + 381 lambda_k_B + 2673 lambda_k_C + 282 lambda_k_D
 + 1608 lambda_k_E + 2074 lambda_k_F + 75 lambda_k_G + 458 lambda_k_H
 + 1722 lambda_k_I + 400 lambda_k_J + 1217 lambda_k_K + 2532 lambda_k_L
 + 1303 lambda_k_M - 381 theta_r <= 0

_C3: 8259 lambda_k_A + 9628 lambda_k_B + 70923 lambda_k_C + 9683 lambda_k_D
 + 40630 lambda_k_E + 47420 lambda_k_F + 7115 lambda_k_G + 10177 lambda_k_H
 + 29124 lambda_k_I + 8987 lambda_k_J + 34680 lambda_k_K + 51536 lambda_k_L
 + 32683 lambda_k_M - 9628 theta_r <= 0

_C4: 23756 lambda_k_A + 24183 lambda_k_B + 163483 lambda_k_C
 + 10370 lambda_k_D + 99047 lambda_k_E + 128635 lambda_k_F + 11962 lambda_k_G
 + 32436 lambda_k_H + 83862 lambda_k_I + 14618 la

C:\Users\duclh3\AppData\Local\Temp\ipykernel_19348\1132472900.py:9: DeprecationWarning: 'indexs' is deprecated; use 'indices'.
  lambda_k = LpVariable.dicts(f'lambda_k', lowBound=0, indexs=K)


('B：0.968\n', 0.96840307)

In [20]:
# VRS_DEA_Model
def getTechnicalEfficiency(r):

    # Model Building 
    model = LpProblem('VRS_model', LpMinimize) # 建立一個新的model，命名為model
    
    # Decision variables Building 
    theta_r = LpVariable(f'theta_r')
    lambda_k = LpVariable.dicts(f'lambda_k', lowBound=0, indexs = K)
    
    # Objective Function setting
    model += theta_r
    
    # Constraints setting
    for i in I:
        model += lpSum([
                lambda_k[k] * X[i][k]
            for k in K]) <= theta_r * float(X[i][K[r]])
    for j in J:
        model += lpSum([
                lambda_k[k] * Y[j][k]
            for k in K]) >= float(Y[j][K[r]])
    model += lpSum([ lambda_k[k] for k in K]) == 1
    
    # model solving 
    model.solve()  
    
    return f'{K[r]}：{round(value(model.objective), 3)}\n', value(model.objective)

In [36]:
print(getOverallEfficiency(20))

C:\Users\duclh3\AppData\Local\Temp\ipykernel_19348\3947460065.py:9: DeprecationWarning: 'indexs' is deprecated; use 'indices'.
  lambda_k = LpVariable.dicts(f'lambda_k', lowBound=0, indexs=K)


IndexError: list index out of range

In [27]:
K[1]

'B'